In [1]:
import os
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec

In [8]:
hype_skill_scores_long = pd.read_csv('./results/hds_nse_skill/skill_scores.csv')

raven_skill_scores_long = pd.read_csv('./results/raven_nse_skill/skill_scores.csv')

gauge_info= pd.read_csv('inputs/gauge_info.csv', index_col=0)

output_dir = './'

# Define periods and corresponding masks
periods = {
    'calibration': 'cal_mask',
    'validation': 'val_mask',
 #   'all': None  # No mask for 'all'
}

period = "validation"

In [6]:

# Create dictionary mapping subbasin_id to Gauge ID
subbasin_to_gauge_hype = gauge_info['hype_subbasin_id'].to_dict()

# --- Create dictionaries mapping subbasin_id → Gauge ID ---
subbasin_to_gauge_hype  = {v: k for k, v in gauge_info['hype_subbasin_id'].to_dict().items()}
subbasin_to_gauge_raven = {v: k for k, v in gauge_info['raven_subbasin_id'].to_dict().items()}

# --- Subset to only validation period ---
df_hype  = hype_skill_scores_long[hype_skill_scores_long["period"] == period].copy()
df_raven = raven_skill_scores_long[raven_skill_scores_long["period"] == period].copy()

# --- Ensure subbasin column is int ---
df_hype["subbasin"]  = df_hype["subbasin"].astype(int)
df_raven["subbasin"] = df_raven["subbasin"].astype(int)

# --- Map gauge IDs ---
df_hype["gauge_id"]  = df_hype["subbasin"].map(subbasin_to_gauge_hype)
df_raven["gauge_id"] = df_raven["subbasin"].map(subbasin_to_gauge_raven)

# --- Make categorical for plotting ---
df_hype["gauge_id"]  = pd.Categorical(df_hype["gauge_id"], categories=df_hype["gauge_id"].unique(), ordered=True)
df_raven["gauge_id"] = pd.Categorical(df_raven["gauge_id"], categories=df_raven["gauge_id"].unique(), ordered=True)


KeyError: 'period'

In [ ]:
# --- Layout: 2 rows (HYPE, Raven) × 2 columns (boxplot, CDF) ---
fig = plt.figure(figsize=(12, 10), dpi=300)
gs = GridSpec(2, 2, width_ratios=[4, 0.7], figure=fig)

datasets = {
    "HYPE": df_hype,
    "Raven": df_raven
}

axes_box, axes_cdf = [], []

for row, (model, df_period) in enumerate(datasets.items()):
    # --- Axes ---
    ax_box = fig.add_subplot(gs[row, 0])
    ax_cdf = fig.add_subplot(gs[row, 1])
    axes_box.append(ax_box)
    axes_cdf.append(ax_cdf)

    # --- BOX PLOT ---
    sns.boxplot(
        x="gauge_id", y="skill_score", data=df_period,
        ax=ax_box, color=box_color, width=0.6, fliersize=3
    )

    # Lowest benchmark
    min_rows = df_period.loc[df_period.groupby("subbasin")["skill_score"].idxmin()]
    x_positions = [np.where(df_period["gauge_id"].cat.categories == gid)[0][0]
                   for gid in min_rows["gauge_id"]]
    ax_box.scatter(x_positions, min_rows["skill_score"],
                   color=lowest_benchmark_color, s=30, zorder=5)

    # Mean flow (NSE benchmark)
    mean_rows = df_period[df_period["benchmark"] == "bm_mean_flow"]
    x_mean_positions = [np.where(df_period["gauge_id"].cat.categories == gid)[0][0]
                        for gid in mean_rows["gauge_id"]]
    ax_box.scatter(x_mean_positions, mean_rows["skill_score"],
                   color=mean_flow_color, s=30, zorder=5, marker="s")

    # Top benchmark
    max_rows = df_period.loc[df_period.groupby("subbasin")["skill_score"].idxmax()]
    x_max_positions = [np.where(df_period["gauge_id"].cat.categories == gid)[0][0]
                       for gid in max_rows["gauge_id"]]
    ax_box.scatter(x_max_positions, max_rows["skill_score"],
                   color=top_benchmark_color, s=30, zorder=5, marker="o")

    # Horizontal line
    ax_box.axhline(0, color="red", linestyle="--", linewidth=1)

    # Y limits
    y_min = df_period["skill_score"].min() - 0.4
    y_max = 1
    ax_box.set_ylim(y_min, y_max)

    # Title / labels
    ax_box.set_title(f"{model} Validation Skill Scores", fontsize=12, weight="bold", pad=45)
    ax_box.set_ylabel("Skill Score", fontsize=10)
    ax_box.set_xlabel("Gauge ID", fontsize=10)
    ax_box.tick_params(axis="x", rotation=45)

    # Little ticks above x-axis labels
    for x in range(len(df_period["gauge_id"].cat.categories)):
        ax_box.vlines(x, ymin=y_min, ymax=y_min + 0.02*(y_max - y_min),
                      color="black", linewidth=0.8)

    # --- CDF PLOT ---
    lowest_vals = min_rows["skill_score"].values
    mean_vals   = mean_rows["skill_score"].values
    top_vals    = max_rows["skill_score"].values
    
    def ecdf(data):
        x = np.sort(data)
        y = np.arange(1, len(x) + 1) / len(x)
        return x, y
    
    for vals, color, label, ls, marker in [
        (lowest_vals, lowest_benchmark_color, "Lowest", "-", "o"),
        (mean_vals, mean_flow_color, "NSE (Mean Flow)", "--", "s"),
        (top_vals, top_benchmark_color, "Top", "-.", "d")
    ]:
        x, y = ecdf(vals)
        ax_cdf.plot(x, y, color=color, linestyle=ls, label=label, linewidth=1)
        ax_cdf.scatter(x, y, color=color, s=10, marker=marker, zorder=5)
    
    ax_cdf.set_title(f"{model} Validation CDF", fontsize=10, weight="bold", pad=10)
    ax_cdf.set_xlabel("Skill Score", fontsize=9)
    ax_cdf.set_ylabel("CDF", fontsize=9)
    ax_cdf.set_ylim(0, 1.02)
    ax_cdf.grid(True, linestyle="--", linewidth=0.5, alpha=0.7)

# --- Shared legend ---
fig.legend(
    handles=[
        plt.Line2D([0], [0], color=lowest_benchmark_color, marker="o", label="Lowest Benchmark"),
        plt.Line2D([0], [0], color=mean_flow_color, linestyle="--", marker="s", label="NSE (Mean Flow)"),
        plt.Line2D([0], [0], color=top_benchmark_color, linestyle="-.", marker="d", label="Top Benchmark")
    ],
    loc="lower center", ncol=3, frameon=False
)

plt.tight_layout(rect=[0, 0.05, 1, 1])

# Save figures
plt.savefig(os.path.join(output_dir, "skill_scores_validation.pdf"), dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(output_dir, "skill_scores_validation.png"), dpi=300, bbox_inches="tight")

plt.show()